In [4]:
import sys 
import gymnasium as gym
from gymnasium.wrappers import FlattenObservation
import kidspuzzles
from stable_baselines3 import A2C

n_digits = 1

env = gym.make('kidspuzzles/DigitsPuzzleEnv-v0', render_mode = 'human', n_digits = n_digits)

model = A2C("MultiInputPolicy", env, verbose=1, device="mps", tensorboard_log=f"./logs/a2c_digitspuzzle_nd{n_digits}/")
model.learn(total_timesteps=10000)

Using mps device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./logs/a2c_digitspuzzle_nd1/A2C_1
--------------------------------------
| rollout/              |            |
|    ep_len_mean        | 9.69       |
|    ep_rew_mean        | 1          |
| time/                 |            |
|    fps                | 3          |
|    iterations         | 100        |
|    time_elapsed       | 138        |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -1.37      |
|    explained_variance | 0.08666366 |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | 0.195      |
|    value_loss         | 0.0278     |
--------------------------------------
--------------------------------------
| rollout/              |            |
|    ep_len_mean        | 7.05       |
|    ep_rew_mean        | 1          |
| time/                 |            |
|  

In [ ]:
import time
vec_env = model.get_env()
observations = vec_env.reset()
reward_sum = 0
for _ in range(10):
    action, _state = model.predict(observations)
    observations, reward, terminated, info = vec_env.step(action)
    vec_env.render("human")
    time.sleep(5)

    reward_sum += reward
    if terminated:
        observations = vec_env.reset()

print("Total reward: ", )

Total reward:  [8.]


In [8]:
env.close()